## 获取数据
 下载数据[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) ，并解压，解压后的数据放在cifar-10-data目录下。

In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import problem_unittests as tests
import numpy as np
import preprocess

cifar10_dataset_folder_path = 'cifar-10-data'

## 预处理
### Normalize

In [2]:
def normalize(x):
    return x/255

### One-hot encode

In [3]:
from sklearn import preprocessing
labels = np.array(range(10))
lb = preprocessing.LabelBinarizer()
lb.fit(labels)

def one_hot_encode(x):
    return lb.transform(x)


## 处理数据
处理所有cifar-10数据，使用10%的训练集数据作为验证集

In [4]:
preprocess.preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)

# Check Point
将预处理后的数据保存至磁盘

In [5]:
import pickle
import problem_unittests as tests
import preprocess

# 加载预处理过的validation data
valid_features, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))

## Build the network

### Input


In [6]:
import tensorflow as tf
tf.reset_default_graph()

def neural_net_image_input(image_shape):
    return tf.placeholder(tf.float32,shape=(None,)+image_shape,name = 'x')


def neural_net_label_input(n_classes):
    return tf.placeholder(tf.float32,shape=(None,n_classes),name = 'y')


def neural_net_keep_prob_input():
    return tf.placeholder(tf.float32,name = 'keep_prob')


### Convolution and Max Pooling Layer

In [7]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    global fw
    global fb
    fw = tf.Variable(tf.truncated_normal(conv_ksize+(x_tensor.get_shape().as_list()[3],)+(conv_num_outputs,)))
    fb = tf.Variable(tf.random_normal([conv_num_outputs]))
    conv_strides = (1,) + conv_strides + (1,)
    pool_ksize = (1,) + pool_ksize + (1,)
    pool_strides = (1,) + pool_strides + (1,)
    padding = 'SAME'
    x_tensor = tf.nn.conv2d(x_tensor,fw,conv_strides,padding)
    x_tensor = tf.nn.bias_add(x_tensor,fb)
    x_tensor = tf.nn.relu(x_tensor)
    x_tensor = tf.nn.max_pool(x_tensor,pool_ksize,pool_strides,padding)
    return x_tensor 


### Flatten Layer

In [8]:
def flatten(x_tensor):
    fi_size = x_tensor.get_shape().as_list()[1]*x_tensor.get_shape().as_list()[2]*x_tensor.get_shape().as_list()[3]
    flt = tf.reshape(x_tensor,[-1,fi_size])
    return flt

### Fully-Connected Layer

In [9]:
def fully_conn(x_tensor, num_outputs):
    wd1 = tf.Variable(tf.random_normal([x_tensor.get_shape().as_list()[1],num_outputs]))
    bd1 = tf.Variable(tf.random_normal([num_outputs]))
    return tf.nn.relu(tf.add(tf.matmul(x_tensor,wd1),bd1))   # -- add nonlinear --


### Output Layer

In [10]:
def output(x_tensor, num_outputs):
    wd2 = tf.Variable(tf.random_normal([x_tensor.get_shape().as_list()[1],num_outputs]))
    bd2 = tf.Variable(tf.random_normal([num_outputs]))
    return tf.add(tf.matmul(x_tensor,wd2),bd2)


### Create Convolutional Model 

In [11]:
def conv_net(x, keep_prob):
    # ===layer 1====
    conv1 = conv2d_maxpool(x,120,(4,4),(2,2),(4,4),(2,2))
    #conv1 = conv2d_maxpool2(conv1,75,(4,4),(2,2),(4,4),(2,2))
    

    #   flatten(x_tensor)
    # ====layer 2====
    conv2 = flatten(conv1)
    #conv2=tf.nn.dropout(conv2,keep_prob)  #----- add dropout ----

    
    #   fully_conn(x_tensor, num_outputs)
    #===layer 3====
    conv3 = fully_conn(conv2,30000)
    conv3 = tf.nn.dropout(conv3,keep_prob)  #----- add dropout ----
    
    
    #====layer 4====
    outputs = output(conv3,10)
    
    return outputs


### Build the Network

In [12]:
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((32, 32, 3))
y = neural_net_label_input(10)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Logits
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


In [13]:
#x.get_shape().as_list()
#y.get_shape().as_list()
#keep_prob.get_shape().as_list()

## Train the Neural Network


###  单次训练

In [14]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={
        x: feature_batch,
        y: label_batch,
        keep_prob: keep_probability
    })


### 打印状态

In [15]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    loss = session.run(cost,feed_dict={
        x: feature_batch,
        y: label_batch,
        keep_prob: 1.0
    })
    valid_acc = sess.run(accuracy,feed_dict={
        x: valid_features,
        y: valid_labels,
        keep_prob: 1.0
    })
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss,valid_acc))
    # pass

### Hyperparameters

In [16]:
epochs = 10
batch_size = 128
keep_probability = 0.5

### Training

In [20]:
save_model_path = './image_classification'

print('Training...')
with tf.Session() as sess:
    # 初始化变量
    sess.run(tf.global_variables_initializer())
    
    # Training
    for epoch in range(epochs):
        n_batches = 5
        for batch_i in range(1, n_batches + 1):
            for batch_features, batch_labels in preprocess.load_preprocess_training_batch(batch_i, batch_size):
                train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
            print_stats(sess, batch_features, batch_labels, cost, accuracy)
            
    # 保存模型
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)


"\nsave_model_path = './image_classification'\n\nprint('Training...')\nwith tf.Session() as sess:\n    # 初始化变量\n    sess.run(tf.global_variables_initializer())\n    \n    # Training\n    for epoch in range(epochs):\n        n_batches = 5\n        for batch_i in range(1, n_batches + 1):\n            for batch_features, batch_labels in preprocess.load_preprocess_training_batch(batch_i, batch_size):\n                train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)\n            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')\n            print_stats(sess, batch_features, batch_labels, cost, accuracy)\n            \n    # 保存模型\n    saver = tf.train.Saver()\n    save_path = saver.save(sess, save_model_path)\n"

# Checkpoint

## 测试模型
在测试集上测试训练的模型.

In [21]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import pickle
import preprocess
import random

try:
    if batch_size:
        pass
except NameError:
    batch_size = 64

save_model_path = './image_classification'
n_samples = 4
top_n_predictions = 3

def test_model():

    test_features, test_labels = pickle.load(open('preprocess_training.p', mode='rb'))
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # 获取Tensors
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
        loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for train_feature_batch, train_label_batch in preprocess.batch_features_labels(test_features, test_labels, batch_size):
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: train_feature_batch, loaded_y: train_label_batch, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))


test_model()

INFO:tensorflow:Restoring parameters from ./image_classification
Testing Accuracy: 0.6541732594936709

